### Import flight table

In [34]:
import psycopg2
import pandas as pd
import APIkey

In [35]:
pwd = APIkey.progre_key["password"]

In [36]:
con = psycopg2.connect(database="mid_term_project", user="lhl_student", password=pwd, host="mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com", port="5432")

In [37]:
sql = 'select fl_date,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_delay,crs_arr_time,arr_delay,cancelled,cancellation_code,crs_elapsed_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay from flights'

In [38]:
#df_flight = pd.read_sql_query(sql,con) 

In [57]:
#df_flight.info()

In [31]:
columns = ['fl_date','op_unique_carrier','tail_num','op_carrier_fl_num','origin_airport_id','dest_airport_id','crs_dep_time','dep_delay','crs_arr_time','arr_delay','cancelled','cancellation_code','crs_elapsed_time','distance','carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay']

### Facts about flights table
total records : 15927485 

total columns: 42
tail_num  is null:     49269
delay : 3009148 


### separate train data with needed features. data preparation. 

In [42]:

df_train = df_flight[['fl_date','op_unique_carrier','tail_num','op_carrier_fl_num','origin_airport_id','dest_airport_id','crs_dep_time','crs_arr_time','crs_elapsed_time','distance']]

In [58]:
df_train.isnull().sum()

fl_date                  0
op_unique_carrier        0
tail_num             49269
op_carrier_fl_num        0
origin_airport_id        0
dest_airport_id          0
crs_dep_time             0
crs_arr_time             0
crs_elapsed_time        20
distance                 0
dtype: int64

In [52]:
df_train.head()

,fl_date,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,distance
0,2018-04-02,WN,N437WN,6585,12892,11259,1605,2105,180.0,1246.0
1,2018-04-02,WN,N706SW,417,12892,11292,2100,15,135.0,862.0
2,2018-04-02,WN,N7819A,624,12892,11292,600,930,150.0,862.0
3,2018-04-02,WN,N969WN,819,12892,11292,1335,1705,150.0,862.0
4,2018-04-02,WN,N400WN,2072,12892,11292,2230,145,135.0,862.0


In [71]:
df_train.nunique()

fl_date               730
op_unique_carrier      28
tail_num             6487
op_carrier_fl_num    7252
origin_airport_id     376
dest_airport_id       376
crs_dep_time         1412
crs_arr_time         1437
crs_elapsed_time      693
distance             1623
dtype: int64

In [72]:
df_train2 = df_train.iloc[:1000000,:]

In [74]:
df_train2.drop(columns=['tail_num','op_carrier_fl_num'],inplace=True)

D:\big data\anaconda3\envs\LH\lib\site-packages\pandas\core\frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [75]:
df_train2.to_csv('df_train2')

## Lable data with all delays, data preparation.

In [66]:
#df_lable['total_delay'] = df_lable['carrier_delay'] + df_lable['weather_delay'] + df_lable['nas_delay'] + df_lable['security_delay'] + df_lable['late_aircraft_delay'] + df_lable['arr_delay'] + df_lable['dep_delay']

In [59]:
def is_delay(x):
    '''
    Generate a column to identify delay.
    input : total_delay
    output : bool 1 is delay, 0 is not.
    '''
    if x > 0:
        return 1
    else: 
        return 0

In [65]:
#df_lable['isdelay'] = df_lable['total_delay'].apply(is_delay)

In [64]:
df_lable['isdelay'].value_counts()

0    12918338
1     3009148
Name: isdelay, dtype: int64

In [69]:
df_lable.describe()

,dep_delay,arr_delay,cancelled,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,total_delay,isdelay
count,1.566373e+07,1.561574e+07,1.592748e+07,3.009484e+06,3.009484e+06,3.009484e+06,3.009484e+06,3.009484e+06,3.009149e+06,1.592749e+07
mean,1.054971e+01,5.475932e+00,1.690210e-02,2.037512e+01,3.674443e+00,1.623685e+01,8.853279e-02,2.729212e+01,1.983783e+02,1.889280e-01
std,4.780636e+01,4.987401e+01,1.289047e-01,6.320810e+01,3.111776e+01,3.775392e+01,3.135871e+00,5.334582e+01,2.627431e+02,3.914514e-01
min,-1.280000e+03,-1.290000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.700000e+01,0.000000e+00
25%,-6.000000e+00,-1.500000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.700000e+01,0.000000e+00
50%,-2.000000e+00,-6.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,3.000000e+00,1.180000e+02,0.000000e+00
75%,7.000000e+00,8.000000e+00,0.000000e+00,1.700000e+01,0.000000e+00,2.000000e+01,0.000000e+00,3.300000e+01,2.330000e+02,0.000000e+00
max,2.976000e+03,2.973000e+03,1.000000e+00,2.973000e+03,2.900000e+03,1.848000e+03,1.078000e+03,2.962000e+03,8.922000e+03,1.000000e+00


In [70]:
df_lable.head(20)

,dep_delay,arr_delay,cancelled,cancellation_code,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,total_delay,isdelay
0,11.0,-1.0,0.0,None,NaN,NaN,NaN,NaN,NaN,NaN,0
1,61.0,47.0,0.0,None,11.0,0.0,0.0,0.0,36.0,155.0,1
2,-4.0,0.0,0.0,None,NaN,NaN,NaN,NaN,NaN,NaN,0
3,29.0,7.0,0.0,None,NaN,NaN,NaN,NaN,NaN,NaN,0
4,18.0,1.0,0.0,None,NaN,NaN,NaN,NaN,NaN,NaN,0
5,25.0,13.0,0.0,None,NaN,NaN,NaN,NaN,NaN,NaN,0
6,5.0,-2.0,0.0,None,NaN,NaN,NaN,NaN,NaN,NaN,0
7,150.0,135.0,0.0,None,99.0,0.0,0.0,0.0,36.0,420.0,1
8,5.0,4.0,0.0,None,NaN,NaN,NaN,NaN,NaN,NaN,0
9,25.0,16.0,0.0,None,2.0,0.0,0.0,0.0,14.0,57.0,1


### Table **flights_test**

total number of records: 660556
total columns: 20
fl_date: 2020-1-1 to 2020-01-31
crs_elapsed_time mean : 144.95
distance mean: 795


In [27]:
sql = 'select * from flights_test limit 100'

In [28]:
df_test = pd.read_sql(sql,con)

In [29]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   fl_date             100 non-null    datetime64[ns]
 1   mkt_unique_carrier  100 non-null    object        
 2   branded_code_share  100 non-null    object        
 3   mkt_carrier         100 non-null    object        
 4   mkt_carrier_fl_num  100 non-null    int64         
 5   op_unique_carrier   100 non-null    object        
 6   tail_num            100 non-null    object        
 7   op_carrier_fl_num   100 non-null    int64         
 8   origin_airport_id   100 non-null    int64         
 9   origin              100 non-null    object        
 10  origin_city_name    100 non-null    object        
 11  dest_airport_id     100 non-null    int64         
 12  dest                100 non-null    object        
 13  dest_city_name      100 non-null    object        


In [30]:
df_test

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363
1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1150,1320,N,90,1,363
2,2020-01-01,WN,WN,WN,4598,WN,N7885A,4598,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",2020,2130,N,70,1,333
3,2020-01-01,WN,WN,WN,4761,WN,N551WN,4761,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",1340,1455,N,75,1,333
4,2020-01-01,WN,WN,WN,5162,WN,N968WN,5162,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",915,1035,N,80,1,333
